In [ ]:
import pandas as pd
from datetime import datetime, date

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Load all data at once

In [ ]:
conditions = pd.read_csv("../data/csv/conditions.csv")
patients = pd.read_csv("../data/csv/patients.csv")
observations = pd.read_csv("../data/csv/observations.csv")
#care_plans = pd.read_csv("../data/csv/careplans.csv")
encounters = pd.read_csv("../data/csv/encounters.csv")
devices = pd.read_csv("../data/csv/devices.csv")
#supplies = pd.read_csv('../data/csv/supplies.csv')
procedures = pd.read_csv("../data/csv/procedures.csv")
medications = pd.read_csv("../data/csv/medications.csv")

In [ ]:
# set your condition, this will be our prediction "target"
condition = 'Chronic congestive heart failure (disorder)'

In [ ]:
# perform any processing that is needed for your features
patients = patients.rename(columns={'Id': 'PATIENT'})

# convert birthdate to datetime
patients['BIRTHDATE'] = pd.to_datetime(patients['BIRTHDATE'])

# calculate age
patients['AGE'] = patients['BIRTHDATE'].apply(lambda x : (datetime.now().year - x.year))

In [ ]:
# set your features (table name, column name)
features = [('patients','AGE'), ('patients','GENDER')]

In [ ]:
# create your dataset
X = patients[['PATIENT']]

for feature in features:
    merge_col = 'PATIENT'
    X = X.merge(locals()[feature[0]][[merge_col, feature[1]]], on=merge_col)
X

In [ ]:
# add your label
X = X.merge(conditions.loc[conditions['DESCRIPTION']==condition][['PATIENT','START']], on='PATIENT', how='left')
X = X.rename(columns={'START': 'label'})
X['label'] = X['label'].notnull()
X